In [1]:
#This file is designed to export gas data of the snapshots into a file that can be used to create plots with gas in the background
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import os
import pickle
import matplotlib
import matplotlib.colors as colors
import gizmo_analysis as gizmo
import astropy
from astropy.io import ascii
#############################################################################
#constants
#############################################################################
MsunToGm = 1.99e33
KpcToCm = 3.086e21
mp = 1.67e-24
#bin_edge = 10.
bin_edge = 30.
bins = np.arange(-25,25,0.1)

In [2]:
############################################################################
#read in sim files and find relevant particles
############################################################################
simname = 'm12f_res7100_mhdcv'
simtype="fire2"  #this is the simtype eg. fire2, fire3, sf-fire3, sf-fire3-alpha01, sf-fire3-alpha03, sf-fire3-alpha05 
simdir = '/scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12f_r7100/mhdcv/1Myr/'
work_data_path="/home1/07428/binod/work2/data/gas_data/"
gas_datapath=work_data_path+simname+"_gas_data_pkl/"  #path to store the gas data
if not os.path.exists(gas_datapath):
    os.makedirs(gas_datapath)
                
#snap=596 #this is the snapshot at which the clusters were taken from using the fof algorithm I am using this to extract information from the snapshot where the clusters were first seen
snapshot_start=588  #snapshot to begin creating the figure
snapshot_end=590    #snapshot to stop at

snapshot_list=np.arange(snapshot_start,snapshot_end+1) #create a list of snapshot numbers to plot to plot eg. [596,597, ...]
time=snapshot_list-snapshot_start #time starts from zero here where t=0 is at the snapshot where we start from eg. [0,1,2,3,....]

#reading in snapshot_times.txt file to get snapshot numbers and times
# columns are: snapshot scale-factor redshift time[Gyr] time_width[Myr]
snapshot_times = simdir + 'snapshot_times.txt'
snaptime_data = astropy.io.ascii.read(snapshot_times, guess=False, comment="#")
snaptime_data = np.genfromtxt(snapshot_times, usecols=(0,3), skip_header=4, dtype=float) #the first and fourth columns are the only ones we need 
snaps = np.array(snaptime_data[:,0]) #col1 = first column saved from text file #This is a collection of all snapshot nos.
times = np.array(snaptime_data[:,1]) #col4 = second column saved #This is a collection of times equivalent to those snapshot nos.
#######################################
#######################################

In [ ]:
tracked_gas_all_snaps={}
for i in range(len(time)):                              
    snapnumber=time[i]+snapshot_start      #this is the true snapshot number each time for eg, 596, 597 .. and son
    snaptime = times[np.where(snaps == snapnumber)][0] #time of snapshot in Gyr
    tracked_gas={} 
    part = gizmo.io.Read.read_snapshots(['all'],'snapshot_index', time[i]+snapshot_start, simulation_name=simname, simulation_directory=simdir, assign_hosts_rotation=True, assign_hosts=True)  
    t = np.max(part['star'].prop('form.time'))  
    
    rGas = part['gas'].prop('host.distance.principal.cylindrical')[:,0]
    zGas = part['gas'].prop('host.distance.principal.cylindrical')[:,1]
    
    xGas = part['gas'].prop('host.distance.principal.cartesian')[:,0]
    yGas = part['gas'].prop('host.distance.principal.cartesian')[:,1]
    zGas = part['gas'].prop('host.distance.principal.cartesian')[:,2]
    
    vxGas = part['gas'].prop('host.velocity.principal.cartesian')[:,0]
    vyGas = part['gas'].prop('host.velocity.principal.cartesian')[:,1]
    vzGas = part['gas'].prop('host.velocity.principal.cartesian')[:,2]
    
    mGas = part['gas']['mass']
    rhoGas = part['gas']['density']
    tGas = part['gas']['temperature']
    idGas = part['gas']['id']
    
    i_gas = np.where((rGas <= bin_edge) & (np.fabs(zGas) <= 1.5) & (part['gas']['density']*((MsunToGm/KpcToCm**3)/mp) >= 10.) & (tGas <= 1e4))
    
    x = xGas[i_gas]
    y = yGas[i_gas]
    z = zGas[i_gas]
    vx = vxGas[i_gas]
    vy = vyGas[i_gas]
    vz = vzGas[i_gas]
    m = mGas[i_gas]
    rho = part['gas'].prop('number.density')[i_gas]
    id = part['gas']['id'][i_gas]
    
    ###########################################################################
    #gas image (2d histogram)
    ###########################################################################
    #cold (< 10^4 K) gas in the midplane (|z| <= 1.5 kpc within bin_edge
    v =  np.where((rGas <= bin_edge) & (np.fabs(zGas) <= 1.5) & (tGas <= 1e4))
    face, xh, yh = np.histogram2d(part['gas'].prop('host.distance.principal.cartesian')[v,1][0],part['gas'].prop('host.distance.principal.cartesian')[v,0][0],bins=[bins,bins], weights=part['gas']['mass'][v])

    ###########################################################################
    tracked_gas={"snaptime":snaptime,"v":v,"face":face,"xh":xh,"yh":yh}
    file_name=simtype+"_gas_data"+str(snapnumber)+".pkl"
    with open(gas_datapath+file_name, 'wb') as output:
        pickle.dump(tracked_gas, output)
    print("\n Stored the gas data for background plot from the snapshot no.",snapnumber,"to filename:",file_name,"\n#####\n")
    '''
    This is how to read the data:
    file_name="gas_data_snapshot_"+str(snapnumber)+".pkl"
    with open(gas_datapath+file_name, "rb") as input:
      import_gasdata = pickle.load(input)
    print(import_gasdata["v"])
    '''
    #tracked_gas_all_snaps.update({snapnumber:tracked_gas})
###########################################
#storing all the gas data from all snapshots as a single file
'''
with open(gas_datapath+simtype+"_gas_data_all_snapshots.pkl", 'wb') as output:
  pickle.dump(tracked_gas_all_snaps, output) #
  
#read it as tracked_gas_all_snaps[snap]["v"]
'''


# in utilities.simulation.Snapshot():
* reading:  scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12f_r7100/mhdcv/1Myr/snapshot_times.txt

  using snapshot index = 588, redshift = 0.008


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12f_r7100/mhdcv/1Myr/output/snapshot_588.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 81363848 particles
    dark2     (id = 2): 7086235 particles
    gas       (id = 0): 64175509 particles
    star      (id = 4): 17775662 particles
    blackhole (id = 5): 0 particles

* reading species: ['dark', 'dark2', 'gas', 'star']
* reading gas properties: ['density', 'electron.fraction', 'hydrogen.neutral.fraction', 'id', 'id.child', 'id.generation', 'magnetic.field', 'mass', 'massfraction', 'position', 'sfr', 'size', 'temperature', 'velocity']
* reading particles from:
    scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12f_r7100/mhdcv/1Myr/output/snapshot_588.hdf

# Test reading gas data

In [1]:
#This file is designed to export gas data of the snapshots into a file that can be used to create plots with gas in the background
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import os
import pickle
import matplotlib
import matplotlib.colors as colors
import gizmo_analysis as gizmo
import astropy
from astropy.io import ascii
from matplotlib import pyplot as plt

#############################################################################
#constants
#############################################################################
MsunToGm = 1.99e33
KpcToCm = 3.086e21
mp = 1.67e-24
#bin_edge = 10.
bin_edge = 30.
bins = np.arange(-25,25,0.1)

In [2]:
!hostname

c455-094.stampede2.tacc.utexas.edu


In [3]:
simname = 'm12f_res7100_mhdcv'
simtype="fire2"  #this is the simtype eg. fire2, fire3, sf-fire3, sf-fire3-alpha01, sf-fire3-alpha03, sf-fire3-alpha05 
simdir = '/scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12f_r7100/mhdcv/1Myr/'
work_data_path="/home1/07428/binod/work2/data/gas_data/"
gas_datapath=work_data_path+simname+"_gas_data_pkl_ll_coordinates/"  #path to store the gas data

In [21]:
def get_gas_plot(v,face,xh,yh,snapnumber):
    
    fig1=plt.figure()
    fig1.set_size_inches(7,7)
    ax=fig1.add_axes([0.17, 0.185, 0.65, 0.65]) #left, bottom, width, height
    bin_edge = 30.
    #bins = np.arange(-5,5,0.1)
    bins = np.arange(-25,25,0.1)
    norm = matplotlib.colors.LogNorm(vmin=1, vmax=1000) #the color range plotted
    im = ax.imshow(face/(((bins[1]-bins[0])*1000)**2),origin='lower',interpolation='nearest',norm=norm,extent=(-25,25,-25,25),cmap='binary') 

    #colorbar for the background gas density
    cmap_gray = matplotlib.cm.get_cmap('binary')
    norm1 = matplotlib.colors.LogNorm(vmin=1,vmax=1000)
    cbar_ax1 = fig1.add_axes([0.04, 0.185, 0.04, 0.64]) # position of gray colorbar (left, bottom, width, height)
    cb1 = fig1.colorbar(im, cax=cbar_ax1, ticklocation='left')
    cb1.set_label('$\Sigma$ (M$_{{\odot}}$/pc$^2$)', labelpad=-5, fontsize=12)
    #plot a scale bar 5 kpc long
    ax.plot([-22.5,-17.5], [22.5,22.5], 'k-', linewidth=5)
    label1 = "5 kpc"
    ax.text(-22.5, 20, label1, fontsize=12.5)
    plt.savefig("./plots/"+str(snapnumber)+".png")
    fig1.close()
    
    
def get_gas_data(file_name):
    
    with open(gas_datapath+file_name, "rb") as input:
        import_gasdata = pickle.load(input)
    
    v=import_gasdata["v"]
    face=import_gasdata["face"]
    xh=import_gasdata["xh"]
    yh=import_gasdata["yh"]
    
    return v,face,xh,yh



    
    

In [ ]:
for snapnumber in range(588,696):

    file_name=simtype+"_gas_data"+str(snapnumber)+".pkl"
    print(file_name)
    get_gas_plot(*get_gas_data(file_name),snapnumber)

#get_gas_plot(v,face,xh,yh)

In [22]:
from PIL import Image
#import glob  #use it if you want to read all certain file type
plot_path="./plots/"    
imgs=[]
for i in range(588,696): #startig and ending value+1 of the index that identifies different file names or imgs = glob.glob("*.png") can do done as well
    filename=str(i)+".png"
    imgs.append(plot_path+filename)
    print("Scanned image from the File: ",filename,end="\r",flush=True)    

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
# Save into a GIF file that loops forever
gifname=simname+".gif"
frames[0].save(plot_path+gifname, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=200, loop=0)    #duration =200 looked good
print("\nSaved the GIF into the File: ",gifname)

Scanned image from the File:  695.png 643.png
Saved the GIF into the File:  m12f_res7100_mhdcv.gif
